In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AdamW
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#Função de tokenização

def tokenize_function(examples):
	return tokenizer(examples['text'], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [6]:
train_testvalid = tokenized_datasets['train'].train_test_split(test_size=0.2)

train_dataset = train_testvalid['train']

valid_dataset = train_testvalid['test']

In [7]:

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

valid_dataloader = DataLoader(valid_dataset, batch_size=8)

In [8]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(DEVICE)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
	output_dir='./results',
	eval_strategy="epoch",
	learning_rate=2e-5,
	per_device_train_batch_size=8,
	per_device_eval_batch_size=8,
	num_train_epochs=3,
	weight_decay=0.01
)

trainer = Trainer(
	model=model,
	args=training_args,
	train_dataset=train_dataset,
	eval_dataset=valid_dataset
)


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.440000,0.426023
2,0.400100,0.471700
3,0.350300,0.543163


AttributeError: 'Trainer' object has no attribute 'evalueate'

In [12]:
metrics = trainer.evaluate()

print(metrics)

{'eval_loss': 0.5431634783744812, 'eval_runtime': 151.7697, 'eval_samples_per_second': 32.945, 'eval_steps_per_second': 4.118, 'epoch': 3.0}


In [13]:
predictions = trainer.predict(valid_dataset)

print(predictions)

PredictionOutput(predictions=array([[ 2.9389307, -2.2080982],
       [ 3.058153 , -2.4002814],
       [ 3.300568 , -2.9272032],
       ...,
       [ 3.2875423, -2.8790765],
       [-2.6376834,  2.1240115],
       [ 3.229237 , -2.7420669]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 0.5431634783744812, 'test_runtime': 151.7261, 'test_samples_per_second': 32.954, 'test_steps_per_second': 4.119})


In [32]:
input_text = "Horrível, nunca assistirei novamente"

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

for key in inputs:
    inputs[key] = inputs[key].to(DEVICE)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

probabilities = F.softmax(logits, dim=-1)

predicted_class = torch.argmax(probabilities).item()

labels = {0: "Negative", 1: "Positive"}
predicted_label = labels[predicted_class]

print(f"Input Text: {input_text}")
print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {probabilities}")

Input Text: Horrível, nunca assistirei novamente
Predicted Label: Negative
Probabilities: tensor([[0.9830, 0.0170]], device='cuda:0')
